## Reimplemented duck simulator

This seems to be working now. There is no assignment with this one except to review this if you are interested. 

### Installation
You need to install mesa and tkinter from the command line before starting jupyter lab.

`pip3 install mesa, tkinter`

### Crucial test

When you run the code cell below, a new window should open with the simulator. If this does not happen, with or without an error message, please let me know. If there is an error message, please send that to me. 

### Some details

1. When Representational mode is on, ducks that currently prefer Source 1 are darkblue and those that prefer Source 2 are cyan. White indicates no preference (all ducks are white when Representational mode is off). There are conditions under which ducks may be white briefly when Representational mode is on. 

2. Food is red. Ducks turn red when they first get food and then their color gradually goes from red to white as they consume the food. If 'Food takes time' is not on, food takes 10 timesteps to consume. Otherwise, the time it takes to consume is equal to the source size. 

3. Restart button starts a new simulation with the current parameters. If you change pond height or width, you have to click Restart for those changes to apply. Note that *image size* can change the pond size while the simulation is running, but it will be cropped if you make it bigger. To make the bigger pond work, click Restart.

4. Continue does just that, from current conditions with current parameters. 

5. Quit button does not currently work. To quit the current simulation, close the simulator window. 

6. To start a new simulation with default parameters, exit the current simulation (close simulator window) and re-run the code cell. 

7. The Representational and Food takes time 'sliders' work like check boxes, just toggling between 0 (off/false) and 1 (on/true). They have immedate effect when you click them, so you can click back and forth between Representational and Nonrepresentational mode.

8. In addition to controlling pond height and width, you can control the dispersal area for each food source with 'Food % Width' and 'Food % Height'. Making these 2 small may lead to a crash/stall, if there are no locations available for food to go. Food width can go from 10% to 90%, literally meaning that is the width of the area relative to pond width. Food height can only go from 10% to 50% because 50% takes you to the midline. You'll see that making Food height bigger makes it more likely that ducks will have time to cross in Nonrepresentational mode (making pond height smaller has a similar effect). 


In [ ]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
import tkinter as tk
from tkinter import Checkbutton
import random


started = False # else it will run immediately
#blocksize = 10 # multiplier determines how large the canvas will be


# Initialize counters for info reports
ducks_above_midline = 0
ducks_below_midline = 0
ducks_crossed_midline = 0
food_count = 0

# # Define global variables for representational and food_takes_time
# # could not get this working properly
# representational_var = True
# food_takes_time_var = False

# # Function to update representational and food_takes_time variables
# could not get this working properly
# def update_checkboxes():
#     global representational_var, food_takes_time_var
#     representational_var = representational_var.get()
#     food_takes_time_var = food_takes_time_var.get()


# Duck Agent
class Duck(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.color = "white"
        self.stay_time = 0
        self.initial_stay_time = 0
        self.switchiness = max(0, min(1, random.gauss((sw_mean_slider.get()/100), (sw_sd_slider.get()/100))))  # Clipped between 0 and 1
        self.preference = random.choice(['source1', 'source2'])  # Initial random preference
        if self.preference == 'source1':
            self.color = 'dark blue'
        elif self.preference == 'source2': 
            self.color = 'cyan'
        else:
            self.color = 'white'


    def update_midline_status(self):
        _, y = self.pos
        self.above_midline = y < self.model.grid.height // 2

    def distance(self, pos1, pos2):
        x1, y1 = pos1
        x2, y2 = pos2
        return ((x2 - x1)**2 + (y2 - y1)**2)**0.5
    
    def get_interpolated_color(self, start_color, end_color, proportion):
        # Unpack RGB values
        r1, g1, b1 = start_color
        r2, g2, b2 = end_color

        # Calculate interpolated RGB values (allows ducks to fade from red to pink to white while eating
        r = int(r1 + proportion * (r2 - r1))
        g = int(g1 + proportion * (g2 - g1))
        b = int(b1 + proportion * (b2 - b1))
    
        return f"#{r:02x}{g:02x}{b:02x}"

    def step(self):
         
        if self.stay_time > 0:
            # Calculate the proportion of stay_time remaining
            proportion = self.stay_time / self.initial_stay_time

            # Define the start and end colors in RGB
            end_color = (255, 50, 50)  # RGB for light red
            start_color = (255, 255, 255)  # RGB for white

            # Calculate the interpolated color
            self.color = self.get_interpolated_color(start_color, end_color, proportion)

            self.stay_time -= 1
            if self.stay_time == 0:
                self.color = "white"
                if self.preference == 'source1':
                    self.color = 'dark blue'
                elif self.preference == 'source2': 
                    self.color = 'cyan'
                else:
                    self.color = 'white'
            return       
        
            # failed code for putting a circle on eating ducks
            # dx, dy = self.pos
            # ox1, oy1 = dx * blocksize, dy * blocksize
            # ox2, oy2 = ox1 + blocksize, oy1 + blocksize
            # canvas.create_oval(ox1, oy1, ox2, oy2, fill="", outline="black")

        # get position
        x, y = self.pos
        # determine if you crossed the midline
        new_above_midline = y < self.model.grid.height // 2
    
        if self.above_midline != new_above_midline:
            self.model.midline_crossings += 1  # Increment the counter in the model

        self.above_midline = new_above_midline
        
        food_list = []  # Initialize to empty list
        relprev1 = self.model.relprev1
        relprev2 = self.model.relprev2

        if representational_slider.get() == 1: # if this is 1 (True) then we are in representational mode
        #if representational_var == 1:
            # Decision-making based on relative prevalence
            # # only consider a switch if we 'get permission' from random value
            if random.uniform(0, 1) < self.switchiness: # consider switching?
                    # choose more prevalent source
                    if random.uniform(0, 1) > max(relprev1, relprev2): 
                        if relprev1 != relprev2: # otherwise retain previous preference because they are the same
                            self.preference = 'source1' if relprev1 > relprev2 else 'source2'
                        elif self.preference == 'either':
                            self.preference = random.choice(['source1', 'source2'])  # choose randomly
                    else:
                        # choose less prevalent source with low probability
                        if random.uniform(0, 1) < min(relprev1, relprev2): # consider switching?
                            if relprev1 != relprev2: # otherwise retain previous preference because they are the same
                                self.preference = 'source1' if relprev1 < relprev2 else 'source2'
                            elif self.preference == 'either':
                                self.preference = random.choice(['source1', 'source2'])  # choose randomly

            if self.preference == 'source1':
                food_list = [(agent, agent.pos) for agent in self.model.schedule.agents if isinstance(agent, Food) and agent.pos is not None and agent.pos[1] < self.model.grid.height // 2]
            elif self.preference == 'source2':
                food_list = [(agent, agent.pos) for agent in self.model.schedule.agents if isinstance(agent, Food) and agent.pos is not None and agent.pos[1] > self.model.grid.height // 2]
        else: # not in representational mode
            self.preference='either'
            # Get all food locations on the grid
            food_list = [(agent, agent.pos) for agent in self.model.schedule.agents if isinstance(agent, Food) and agent.pos is not None]

        #print(f'duck {self.unique_id}: relprev1 {relprev1} relprev2 {relprev2}, pref {self.preference}, color {self.color}')

        if self.preference == 'source1':
            self.color = 'dark blue'
        elif self.preference == 'source2': 
            self.color = 'cyan'
        else:
            self.color = 'white'
 

        # If there is food, find the closest one
        #print(food_list)
        if food_list:
            closest_food_pos = min(food_list, key=lambda x: self.distance(self.pos, x[1]))[1]

            # If the duck is right next to the closest food, move directly into that cell
            neighbors = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
            if closest_food_pos in neighbors:
                self.model.grid.move_agent(self, closest_food_pos)
                agents = self.model.grid.get_cell_list_contents([closest_food_pos])
                for agent in agents:
                    if isinstance(agent, Food):
                        self.color = "red"
                        if food_takes_time_slider.get() == 1:
                        #if food_takes_time_var == 1:
                            self.initial_stay_time = agent.size  # set initial stay_time which we will use to morph color
                            self.stay_time = agent.size
                        else:
                            self.initial_stay_time = 10  # set initial stay_time which we will use to morph color
                            self.stay_time = 10
                        self.model.grid.remove_agent(agent)
                        return

            # Otherwise, find the empty neighbor closest to the food and move there
            empty_neighbors = [pos for pos in neighbors if self.model.grid.is_cell_empty(pos)]
            if empty_neighbors:
                move_towards_food = min(empty_neighbors, key=lambda x: self.distance(closest_food_pos, x))
                self.model.grid.move_agent(self, move_towards_food)

        else:
            # If there is no food, move randomly
            neighbors = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
            empty_neighbors = [pos for pos in neighbors if self.model.grid.is_cell_empty(pos)]
            if empty_neighbors:
                new_position = random.choice(empty_neighbors)
                self.model.grid.move_agent(self, new_position)

# Food Agent
class Food(Agent):
    def __init__(self, unique_id, model, size):
        super().__init__(unique_id, model)
        self.color = "red"
        self.size = size  

class DuckPond(Model):
    def __init__(self, width, height, nducks):
        
        self.current_id = 0  # Add this line to initialize the ID counter
        
        self.num_agents = nducks
        
        #self.grid = MultiGrid(width, height, True)
        # Set torus to False so ducks can't 'wrap' from edges
        self.grid = MultiGrid(width, height, False)  
        
        self.midline_crossings = 0
 
        self.total_food_source1 = 0
        self.total_food_source2 = 0
        self.relprev1 = 0
        self.relprev2 = 0


        self.schedule = RandomActivation(self)
        self.rate1 = rate1_slider.get()
        self.rate2 = rate2_slider.get()

        # Initialize ducks
        for i in range(self.num_agents):
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            while not self.grid.is_cell_empty((x, y)):
                x = random.randrange(self.grid.width)
                y = random.randrange(self.grid.height)
            duck = Duck(self.next_id(), self)  # Use next_id() function here
            self.grid.place_agent(duck, (x, y))
            duck.update_midline_status()  
            self.schedule.add(duck)

    def step(self):
        self.rate1 = rate1_slider.get()
        self.rate2 = rate2_slider.get()
        self.schedule.step()

        food_height = food_height_slider.get()
        use_food_height = int((food_height / 100) * height_slider.get())


        if self.schedule.time % self.rate1 == 0:
            self.add_food(0, use_food_height - 1, size1_slider.get())
            self.total_food_source1 += size1_slider.get()
            
        if self.schedule.time % self.rate2 == 0:
            self.add_food(self.grid.height - use_food_height, self.grid.height - 1, size2_slider.get())
            self.total_food_source2 += size2_slider.get()
            
        # Update prevalence
        prevalence1 = self.total_food_source1 / (self.schedule.time + 1)
        prevalence2 = self.total_food_source2 / (self.schedule.time + 1)

        if prevalence1 + prevalence2 != 0:
            self.relprev1 = prevalence1 / (prevalence1 + prevalence2)
            self.relprev2 = prevalence2 / (prevalence1 + prevalence2)
        else:
            self.relprev1 = 0
            self.relprev2 = 0

    def add_food(self, row_start, row_end, value):
        food_width = food_width_slider.get()        
        use_food_width = int((food_width / 100) * width_slider.get())
        half_width = use_food_width // 2  # Calculate half of the food_width
        start_col = int((self.grid.width // 2) - half_width)  # Calculate the starting column
        end_col = int((self.grid.width // 2) + half_width)  # Calculate the ending column

        x = random.randint(start_col, end_col)  # Randomly pick a column between start_col and end_col
        y = random.randint(row_start, row_end)  # Randomly pick a row between row_start and row_end

        tries = 0
        maxtries = 100
        while not self.grid.is_cell_empty((x, y)) & tries < maxtries:
            x = random.randint(start_col, end_col)
            y = random.randint(row_start, row_end)
            tries += 1

        food = Food(self.next_id(), self, value)  # Create a new Food agent with the value
        self.grid.place_agent(food, (x, y))  # Place the Food agent on the grid
        self.schedule.add(food)  # Add the Food agent to the schedule
        

    def bad_add_food(self, row_start, row_end, size):
        x = random.randint(0, self.grid.width - 1)
        y = random.randint(row_start, row_end)
        while not self.grid.is_cell_empty((x, y)):
            x = random.randint(0, self.grid.width - 1)
            y = random.randint(row_start, row_end)
        food = Food(self.next_id(), self, size)
        self.grid.place_agent(food, (x, y))
        self.schedule.add(food)
  
    def old_add_food(self, row_start, row_end):
        size1 = size1_slider.get()
        size2 = size2_slider.get()
        x = random.randint(0, self.grid.width - 1)
        y = random.randint(row_start, row_end)
        while not self.grid.is_cell_empty((x, y)):
            x = random.randint(0, self.grid.width - 1)
            y = random.randint(row_start, row_end)
        food = Food(self.next_id(), self)  # Use next_id() function here
        self.grid.place_agent(food, (x, y))
        self.schedule.add(food)

    def next_id(self):  # Add this function to generate the next ID
        self.current_id += 1
        return self.current_id

# Continue the simulation
def continue_simulation():
    global max_timesteps  # Declare max_timesteps as global so that we can modify it
    max_timesteps += time_slider.get()  # Increase the maximum allowed timesteps
    root.after(speed_slider.get(), update)  # Continue the simulation
    
# GUI using Tkinter
def update():
    if model.schedule.time < max_timesteps:  # Check if we've reached the maximum timestep
        model.step()
        draw_pond()
        root.after(speed_slider.get(), update)
    else:
        print("Simulation finished.")
        
# Add a Start process for Start button
def start_simulation():
    global started
    started = True
    update()
    
# If we do not quit, we have to restart terminal
def quit_simulation():
    global started
    if not started:
        # if mainloop not running quit makes it hang
        root.mainloop()
    # root.quit()    
    root.destroy()  # this will remove the root from the screen


def draw_pond():
    canvas.delete("all")
    
    # Counters for ducks and other statistics
    ducks_above_midline = 0
    ducks_below_midline = 0
    ducks_crossed_midline = 0  # You'll need to keep track of their previous positions to implement this
    total_ducks = 0
    total_food = 0
    blocksize = blocksize_slider.get()
    midline = model.grid.height // 2
    
    for agents, x, y in model.grid.coord_iter():
        color = "light blue"  # default color
        for agent in agents:
            if isinstance(agent, Duck):
                total_ducks += 1
                if y < midline:
                    ducks_above_midline += 1
                else:
                    ducks_below_midline += 1
                color = agent.color
            elif isinstance(agent, Food):
                total_food += 1
                color = agent.color
                
        canvas.create_rectangle(x*blocksize, y*blocksize, x*blocksize+blocksize, y*blocksize+blocksize, fill=color)
    
    # Draw gridlines
    for i in range(0, model.grid.width * blocksize, blocksize):
        canvas.create_line(i, 0, i, model.grid.height * blocksize, fill="light grey")
    for i in range(0, model.grid.height * blocksize, blocksize):
        canvas.create_line(0, i, model.grid.width * blocksize, i, fill="light grey")
                 
    # Draw black line at the vertical midline
        midline_y = (model.grid.height * blocksize) // 2
        canvas.create_line(0, midline_y, model.grid.width * blocksize, midline_y, fill="black")
    
    # Calculate predictions
    total_source1 = rate1_slider.get() * 5  # Assuming size1 is 5 rows
    total_source2 = rate2_slider.get() * 5  # Assuming size2 is 5 rows
    total_sources = total_source1 + total_source2
    
    predicted_ducks_source1 = (total_source1 / total_sources) * total_ducks
    predicted_ducks_source2 = (total_source2 / total_sources) * total_ducks
    
    # Update statistics
    ducks_above_midline = sum(1 for agent in model.schedule.agents if isinstance(agent, Duck) and agent.pos[1] < model.grid.height // 2)
    ducks_below_midline = sum(1 for agent in model.schedule.agents if isinstance(agent, Duck) and agent.pos[1] >= model.grid.height // 2)
    proportion_above = ducks_above_midline / duck_slider.get()
    proportion_below = ducks_below_midline / duck_slider.get()
    
    
    #food_count = sum(1 for agent in model.schedule.agents if isinstance(agent, Food))
    food_count = sum(1 for _, x, y in model.grid.coord_iter() if any(isinstance(agent, Food) for agent in model.grid.get_cell_list_contents([(x, y)])))


    # Update Labels
    label_ducks_nearer.config(text=f"Ducks nearer: source 1 = {ducks_above_midline} ({proportion_above:.3f}), source 2 = {ducks_below_midline} ({proportion_below:.3f})")
    #label_ducks_crossed.config(text=f"Ducks crossed midline: {ducks_crossed_midline}")
    label_food_count.config(text=f"Food count: {food_count}")
    label_timestep.config(text=f"Current Timestep: {model.schedule.time}")
    label_ducks_crossed['text'] = f"Midline Crossings: {model.midline_crossings}"
    #label_check_states.config(text=f"Representational: {representational_var}   Food takes time: {food_takes_time_var}")
    label_check_states.config(text=f"Representational: {representational_slider.get()}   Food takes time: {food_takes_time_slider.get()}")
    
    # Calculate food prevalence
    total_food_prevalence = model.total_food_source1 + model.total_food_source2
    prevalence1_ratio = model.total_food_source1 / total_food_prevalence if total_food_prevalence > 0 else 0
    prevalence2_ratio = model.total_food_source2 / total_food_prevalence if total_food_prevalence > 0 else 0

    label_food_prevalence.config(text=f"Food Prevalence: Source 1 = {round(prevalence1_ratio, 2)}, Source 2 = {round(prevalence2_ratio, 2)}")

    # Calculate duck preferences
    ducks_prefer_source1 = sum(1 for agent in model.schedule.agents if isinstance(agent, Duck) and agent.preference == 'source1')
    ducks_prefer_source2 = sum(1 for agent in model.schedule.agents if isinstance(agent, Duck) and agent.preference == 'source2')
    total_ducks = ducks_prefer_source1 + ducks_prefer_source2

    preference1_ratio = ducks_prefer_source1 / total_ducks if total_ducks > 0 else 0
    preference2_ratio = ducks_prefer_source2 / total_ducks if total_ducks > 0 else 0

    label_duck_preference.config(text=f"Duck Preferences: Source 1 = {preference1_ratio:.3f}, Source 2 = {preference2_ratio:.3f}")
    
    disc1 = prevalence1_ratio - proportion_above
    disc2 = prevalence2_ratio - proportion_below
    
    disctotal = abs(disc1) + abs(disc2)
    label_discrepancies.config(text=f"Discrepancy: Total={disctotal:.3f}, Src1={disc1:.3f}, Src2 = {disc2:.3f}")
    
def update_model_parameters():
    global model
    model.rate1 = rate1_slider.get()
    model.rate2 = rate2_slider.get()
    model.size1 = size1_slider.get()
    model.size2 = size2_slider.get()
    model.grid.width = width_slider.get()
    model.grid.height = height_slider.get()
    model.num_agents = duck_slider.get()
    
def start_simulation():
    
    # Initialize Model
    model = DuckPond(width_slider.get(), height_slider.get(), duck_slider.get())

    # # Draw initial state
    # draw_pond()

    # Start the simulation
    started = True
    root.after(speed_slider.get(), update)

def restart_simulation():
    global model, max_timesteps, started  # Declare these as global variables so you can modify them

    # Stop the ongoing simulation
    started = False
    blocksize = blocksize_slider.get()
    animation_speed = speed_slider.get()
    
    # Re-initialize the model
    max_timesteps = time_slider.get()
    model = DuckPond(width_slider.get(), height_slider.get(), duck_slider.get())

    # Redraw the pond
    canvas.config(width=blocksize * width_slider.get(), height=blocksize * height_slider.get())
    draw_pond()
    start_simulation()


# Frame for sliders and button
root = tk.Tk()
root.title("Duck Pond Simulation")

# Frame for sliders
frame_sliders = tk.Frame(root)
frame_sliders.pack(side=tk.RIGHT)
#frame_sliders.grid_columnconfigure(1, weight=4)  # Make column 1 (sliders) wider -- does not work


# Frame for buttons
frame_buttons = tk.Frame(root)
frame_buttons.pack(side=tk.LEFT)

# Frame for labels
frame_labels = tk.Frame(root)
frame_labels.pack(side=tk.BOTTOM)

sliderlength = 200

# Use the columnconfigure method to make specific columns wider
frame_sliders.grid_columnconfigure(0, weight=3)  # Make column 0 (empty space) wider
frame_sliders.grid_columnconfigure(1, weight=3)  # Make column 1 (sliders) wider, adjust the weight as needed
frame_sliders.grid_columnconfigure(2, weight=3)  # Make column 2 (empty space) wider
frame_sliders.grid_columnconfigure(3, weight=3)  # Make column 2 (empty space) wider

# Timesteps and Number of Ducks
time_slider = tk.Scale(frame_sliders, from_=10, to=1000, label="Max Timesteps", orient=tk.HORIZONTAL, length=sliderlength)
time_slider.grid(row=0, column=0, columnspan=2 )
time_slider.set(200)

duck_slider = tk.Scale(frame_sliders, from_=10, to=200, label="Number of Ducks", orient=tk.HORIZONTAL, length=sliderlength)
duck_slider.grid(row=0, column=2, columnspan=2)
duck_slider.set(30)

# Pond Height and Pond Width
height_slider = tk.Scale(frame_sliders, from_=20, to=100, label="Pond Height", orient=tk.HORIZONTAL, length=sliderlength)
height_slider.grid(row=1, column=0, columnspan=2)
height_slider.set(30)

width_slider = tk.Scale(frame_sliders, from_=20, to=100, label="Pond Width", orient=tk.HORIZONTAL, length=sliderlength)
width_slider.grid(row=1, column=2, columnspan=2)
width_slider.set(30)

# food height and width
food_width_slider = tk.Scale(frame_sliders, from_=10, to=90, label="Food % Width", orient=tk.HORIZONTAL, length=sliderlength)
food_width_slider.grid(row=2, column=0, columnspan=2)
food_width_slider.set(50)

food_height_slider = tk.Scale(frame_sliders, from_=10, to=50, label="Food % Height", orient=tk.HORIZONTAL, length=sliderlength)
food_height_slider.grid(row=2, column=2, columnspan=2)
food_height_slider.set(20)


# Rate 1 and Size 1
rate1_slider = tk.Scale(frame_sliders, from_=1, to=20, label="Rate 1", orient=tk.HORIZONTAL, length=sliderlength)
rate1_slider.grid(row=3, column=0, columnspan=2)
rate1_slider.set(2)

size1_slider = tk.Scale(frame_sliders, from_=1, to=80, label="Size 1", orient=tk.HORIZONTAL, length=sliderlength)
size1_slider.grid(row=3, column=2, columnspan=2)
size1_slider.set(20)

# Rate 2 and Size 2
rate2_slider = tk.Scale(frame_sliders, from_=1, to=20, label="Rate 2", orient=tk.HORIZONTAL, length=sliderlength)
rate2_slider.grid(row=4, column=0, columnspan=2)
rate2_slider.set(4)

size2_slider = tk.Scale(frame_sliders, from_=1, to=80, label="Size 2", orient=tk.HORIZONTAL, length=sliderlength)
size2_slider.grid(row=4, column=2, columnspan=2)
size2_slider.set(20)

# switchiness mean and sd sliders
sw_mean_slider = tk.Scale(frame_sliders, from_=1, to=99, label="Switchiness mean", orient=tk.HORIZONTAL, length=sliderlength)
sw_mean_slider.grid(row=6, column=0, columnspan=2)
sw_mean_slider.set(50)

sw_sd_slider = tk.Scale(frame_sliders, from_=1, to=99, label="Switchiness standard deviation", orient=tk.HORIZONTAL, length=sliderlength)
sw_sd_slider.grid(row=6, column=2, columnspan=2)
sw_sd_slider.set(20)

# representation and food_takes_time sliders
representational_slider = tk.Scale(frame_sliders, from_=0, to=1, label="Representational", orient=tk.HORIZONTAL, length= sliderlength * .6)
representational_slider.grid(row=8, column=0)
representational_slider.set(1)

food_takes_time_slider = tk.Scale(frame_sliders, from_=0, to=1, label="Food takes time", orient=tk.HORIZONTAL, length=sliderlength * .6)
food_takes_time_slider.grid(row=8, column=2)
food_takes_time_slider.set(0)

# # Add checkboxes for representational_slider and food_takes_time_slider
# could not get this working properly
# representational_var = tk.IntVar()
# representational_checkbox = Checkbutton(frame_sliders, text="Representational", variable=representational_var)
# representational_checkbox.grid(row=7, column=0)
# representational_var.set(1)  # Set the initial state to checked (1)

# food_takes_time_var = tk.IntVar()
# food_takes_time_checkbox = Checkbutton(frame_sliders, text="Food takes time", variable=food_takes_time_var)
# food_takes_time_checkbox.grid(row=7, column=1)
# food_takes_time_var.set(0)  # Set the initial state to unchecked (0)

# # Bind the update_checkboxes function to the checkboxes
# representational_checkbox.config(command=update_checkboxes)
# food_takes_time_checkbox.config(command=update_checkboxes)


# blocksize multiplier
blocksize_slider = tk.Scale(frame_sliders, from_=5, to=50, label="Image size", orient=tk.HORIZONTAL, length=sliderlength)
blocksize_slider.grid(row=7, column=0, columnspan=2)
blocksize_slider.set(20)
# animation speed slider
speed_slider = tk.Scale(frame_sliders, from_=5, to=500, label="Speed (bigger=slower)", orient=tk.HORIZONTAL, length=sliderlength)
speed_slider.grid(row=7, column=2, columnspan=2)
speed_slider.set(100)


# quit button 
quit_button = tk.Button(frame_buttons, text="Quit", command=quit_simulation)
quit_button.grid(row=7, column=0, columnspan=2)


# # start button
# start_button = tk.Button(frame_buttons, text="Start", command=start_simulation)
# start_button.grid(row=1, columnspan=2)

# Continue Button
continue_button = tk.Button(frame_buttons, text="Continue", command=continue_simulation)
continue_button.grid(row=2, columnspan=2)

# Restart button
restart_button = tk.Button(frame_buttons, text="Restart", command=restart_simulation)
restart_button.grid(row=0, columnspan=2)

# Establish the labels that we use to print messages
label_ducks_nearer = tk.Label(frame_labels, text="")
label_ducks_nearer.grid(row=4, column=0, columnspan=1)
label_ducks_crossed = tk.Label(frame_labels, text="")
label_ducks_crossed.grid(row=8, column=0, columnspan=2)
label_food_count = tk.Label(frame_labels, text="")
label_food_count.grid(row=9, column=0, columnspan=2)
# Add a label for displaying the current timestep
label_timestep = tk.Label(frame_labels, text="")
label_timestep.grid(row=10, column=0, columnspan=2)

label_food_prevalence = tk.Label(frame_labels, text="")
label_food_prevalence.grid(row=3, column=0, columnspan=2)

label_duck_preference = tk.Label(frame_labels, text="")
label_duck_preference.grid(row=5, column=0, columnspan=2)
label_discrepancies= tk.Label(frame_labels, text="")
label_discrepancies.grid(row=6, column=0, columnspan=2)

label_check_states = tk.Label(frame_labels, text="")
label_check_states.grid(row=7, column=0, columnspan=2)


# Canvas
blocksize = blocksize_slider.get()
canvas = tk.Canvas(root, bg="white", height=blocksize*height_slider.get(), width=blocksize*width_slider.get())
canvas.pack()

# Initialize max_timesteps here
max_timesteps = time_slider.get()

# # Initialize Model
model = DuckPond(width_slider.get(), height_slider.get(), duck_slider.get())


# Draw initial state
draw_pond()

# call pdate function
if started:
    root.after(speed_slider.get(), update)

# Start the Tkinter main loop
root.mainloop()

Simulation finished.
Simulation finished.
Simulation finished.
